# Testing Out Neural Network Regression

## Importing Necessary Libraries

In [198]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Loading in the Dataset

In [199]:
csv_url = 'https://gist.github.com/NickyDCFP/d675b176350f6a1d54455ffc35e350f9/raw/' + \
          '220fdbc2aec82b8b3e33681e4465813c02b5fccf/Spotify_Youtube.csv'
csv_df = pd.read_csv(csv_url)
# This column may not be necessary (duplicate of index), may remove
csv_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
csv_df.head()

,ID,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,...,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
0,0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,...,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,...,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,...,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,...,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [200]:
irrelevant_columns = ["Url_spotify", "Uri", "Url_youtube", "ID", "Artist", "Track", "Album", "Album_type", "Title", "Channel", "Description", 'Licensed', 'official_video'] # For now, dropping all non-numerical columns
df = csv_df.drop(irrelevant_columns, axis=1)
df.head()

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Comments,Stream
0,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,693555221.0,6220896.0,169907.0,1.040235e+09
1,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,72011645.0,1079128.0,31003.0,3.100837e+08
2,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,8435055.0,282142.0,7399.0,6.306347e+07
3,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,211754952.0,1788577.0,55229.0,4.346636e+08
4,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,618480958.0,6197318.0,155930.0,6.172597e+08


In [201]:
y_cols = ['Views', 'Likes', 'Comments']
# df = df.dropna(subset=y_cols), 
# dropping all nan values as a stopgap for now
df = df.dropna()
df = df.loc[df['Views'] != 0]
num_likes = np.array(df['Likes']).astype(int)
num_comments = np.array(df['Comments']).astype(int)
num_views = np.array(df['Views']).astype(int)

In [202]:
X_df = df.drop(y_cols, axis=1)
X = np.array(X_df).astype('float32')
y = (num_likes / num_views).astype('float32') # Can also use comment proportion as target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)

(14661, 12)


In [203]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [208]:
model = keras.Sequential([
    keras.layers.Dense(3, activation='relu', input_shape=(X_train.shape[1], )),
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(1)
]) # Initial shape that I tried was this, but layers had 15 neurons per, and it was just all over the place, insane loss

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [209]:
fit = model.fit(X_train, y_train,
                epochs=15,
                batch_size=150)

Epoch 1/15
98/98 [==============================] - 1s 859us/step - loss: 1.3560e-04
Epoch 2/15
98/98 [==============================] - 0s 929us/step - loss: 1.2775e-04
Epoch 3/15
98/98 [==============================] - 0s 971us/step - loss: 1.2777e-04
Epoch 4/15
98/98 [==============================] - 0s 915us/step - loss: 1.2796e-04
Epoch 5/15
98/98 [==============================] - 0s 879us/step - loss: 1.2802e-04
Epoch 6/15
98/98 [==============================] - 0s 864us/step - loss: 1.2789e-04
Epoch 7/15
98/98 [==============================] - 0s 910us/step - loss: 1.2776e-04
Epoch 8/15
98/98 [==============================] - 0s 948us/step - loss: 1.2793e-04
Epoch 9/15
98/98 [==============================] - 0s 1ms/step - loss: 1.2800e-04
Epoch 10/15
98/98 [==============================] - 0s 1ms/step - loss: 1.2769e-04
Epoch 11/15
98/98 [==============================] - 0s 1ms/step - loss: 1.2785e-04
Epoch 12/15
98/98 [==============================] - 0s 860us/step - 

In [211]:
loss = model.evaluate(X_test, y_test)
print(f'MSE: {loss}')

153/153 [==============================] - 0s 710us/step - loss: 1.2358e-04
MSE: 0.000123578094644472


In [207]:
from sklearn.feature_selection import mutual_info_regression
scores = mutual_info_regression(X_train_scaled, y_train)
for score in scores:
    print(f"Feature has a score of {score}")

Feature has a score of 0.10152941453915076
Feature has a score of 0.10842275882522134
Feature has a score of 0.07977329548077083
Feature has a score of 0.10626124015819638
Feature has a score of 0.13386115201834592
Feature has a score of 0.12892709716856388
Feature has a score of 0.06848008220028756
Feature has a score of 0.10105367788587927
Feature has a score of 0.11813432275050584
Feature has a score of 0.1162840762148809
Feature has a score of 0.16890972275985394
Feature has a score of 0.18380444622183134


In [212]:
y

array([0.00896958, 0.01498547, 0.03344874, ..., 0.00922965, 0.01347008,
       0.01565247], dtype=float32)